<a href="https://colab.research.google.com/github/Stefano-t/bioinf-lab/blob/main/prediction_ae_hg38.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
print ("downloading epigenomic_dataset")
%pip install epigenomic_dataset &> /dev/null && echo "epigenomic_dataset installed!"

downloading epigenomic_dataset
epigenomic_dataset installed!


In [5]:
from epigenomic_dataset import load_epigenomes
from sklearn.impute import KNNImputer
import pandas as pd
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm.auto import tqdm

In [6]:
# load the dataset

X, y = load_epigenomes(
    cell_line = "K562",
    dataset = "fantom",
    region = "enhancers",
    window_size = 256,
    root = "datasets"
)

In [7]:
# the strand in the enhancer is not relevant
# the columns have values which is equal to the average times the protein 
# apperers in the given window_size 
X

chrom                              SMAD5  NCOA2  ZNF23  ...  NCOR1  NCOA1  KLF1
chrom chromStart chromEnd  strand                       ...                    
chr10 100006381  100006637 .        0.90   0.90   1.37  ...   2.38   0.82  0.69
      100008146  100008402 .        0.73   1.06   0.74  ...   3.11   0.49  1.05
      100014418  100014674 .        0.77   0.33   0.53  ...   0.61   0.97  0.96
      100020216  100020472 .        0.87   0.80   0.83  ...   0.57   0.03  1.07
      100043528  100043784 .        0.64   0.53   0.03  ...   1.26   1.73  0.33
...                                  ...    ...    ...  ...    ...    ...   ...
chrY  7520247    7520503   .        0.00   0.00   0.00  ...   0.00   0.00  0.00
      7724272    7724528   .        0.00   0.00   0.00  ...   0.00   0.00  0.00
      7770029    7770285   .        0.00   0.00   0.00  ...   0.00   0.00  0.00
      7796295    7796551   .        0.00   0.00   0.00  ...   0.00   0.00  0.00
      8007562    8007818   .        0.00   0.00   0.00  ...   0.00   0.00  0.00

[63285 rows x 429 columns]

In [8]:
y

K562
chrom chromStart chromEnd  strand      
chr10 100006381  100006637 .          0
      100008146  100008402 .          0
      100014418  100014674 .          0
      100020216  100020472 .          0
      100043528  100043784 .          0
...                                 ...
chrY  7520247    7520503   .          0
      7724272    7724528   .          0
      7770029    7770285   .          0
      7796295    7796551   .          0
      8007562    8007818   .          0

[63285 rows x 1 columns]

count the NaNs

In [9]:
print(f"The total number of nan is: {X.isna().sum().sum()}")
print(f"Max nan in cols: {X.isna().sum().max()}")

The total number of nan is: 102
Max nan in cols: 93


In [11]:
imputer = KNNImputer() # default to 5 neighbours
# fit_transform creates an nparray, so it's necessary to convert back to DataFrame
X = pd.DataFrame(
    imputer.fit_transform(X),
    index = X.index,
    columns = X.columns
)

scaling the values

`RobustScaler` scales values in range of 10-90 percentiles.

In [12]:
scaler = RobustScaler()
scaler.fit_transform(X)

array([[ 0.31578947,  0.43902439,  1.03278689, ...,  2.14666667,
         0.08333333, -0.32142857],
       [ 0.01754386,  0.82926829,  0.        , ...,  3.12      ,
        -0.46666667,  0.32142857],
       [ 0.0877193 , -0.95121951, -0.3442623 , ..., -0.21333333,
         0.33333333,  0.16071429],
       ...,
       [-1.26315789, -1.75609756, -1.21311475, ..., -1.02666667,
        -1.28333333, -1.55357143],
       [-1.26315789, -1.75609756, -1.21311475, ..., -1.02666667,
        -1.28333333, -1.55357143],
       [-1.26315789, -1.75609756, -1.21311475, ..., -1.02666667,
        -1.28333333, -1.55357143]])

now the Feature Selection procedure should be done.

but we continue without it.

In [42]:
# method to evaluate the models

from typing import Dict, List
from sanitize_ml_labels import sanitize_ml_labels
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score

def evaluate_model_prediction(
    y_true: np.ndarray,
    y_pred: np.ndarray
) -> Dict[str, float]:
  """Return the evaluation of the given predictions.

  Parameters
  -----------------------------
  y_true: np.ndarray,
    the ground truth labels.
  y_pred: np.ndarray,
    the predicted labels.
  
  Raises
  -----------------------------
  ValueError,
    if the two given array do no have the same shape.

  Returns
  -----------------------------
  Dictionary with the performance metrics.
  """
  y_true = y_true.flatten()
  y_pred = y_pred.flatten()
  if y_true.shape != y_pred.shape:
    raise ValueError(
        "The two arrays do not have the same shape: {} != {}".format(
            y.true.shape, y_pred.shape
        )
    )

  float_metrics = average_precision_score, roc_auc_score
  int_metrics = (accuracy_score, ) # tuple of one elem
  return {
      ** { # `**` unwrap the dict as a linear sequence of key-value pairs
          sanitize_ml_labels(metric.__name__): metric(y_true, y_pred)
          for metric in float_metrics
      },
      ** {
          sanitize_ml_labels(metric.__name__): metric(
              y_true, 
              np.round(y_pred).astype(int)
          )
          for metric in int_metrics
      }
  }

In [43]:
def all_model_evalutation(
    y_train_true: np.ndarray,
    y_train_pred: np.ndarray,
    y_test_true: np.ndarray,
    y_test_pred: np.ndarray,
    model_name: str
) -> List[Dict]:

  info = {
      "model_name": model_name
  }

  return [
      {
        ** evaluate_model_prediction(y_train_true, y_train_pred),
        "run_type": "train",
        ** info
      },
      {
        ** evaluate_model_prediction(y_test_true, y_test_pred),
        "run_type": "test",
        ** info  
      },
  ]

In [47]:
from sklearn.tree import DecisionTreeClassifier
# caching layer to speed up execution
from cache_decorator import Cache

# TODO: continue caching layer

@Cache(
    cache_path="active_enhancers/{function_name}/{_hash}.json",

)
def train_decision_tree(
    X_train: np.ndarray, 
    X_test: np.ndarray, 
    y_train: np.ndarray,
    y_test: np.ndarray,
    holdout_number: int
) -> Dict[str, float]:
  tree = DecisionTreeClassifier(
      max_depth = 10,
      min_samples_leaf = 50
  )
  tree.fit(X_train,y_train)
  y_train_pred = tree.predict(X_train)
  y_test_pred = tree.predict(X_test)
  return all_model_evalutation(
      y_train, 
      y_train_pred,
      y_test,
      y_test_pred,
      "Decision tree"
  )

In [30]:
number_of_splits = 10

holdouts_generator = StratifiedShuffleSplit(
  n_splits = number_of_splits,
  test_size = 0.2, # 20% of the dataset as test set
)

In [44]:
all_performance = []

for train_indices, test_indices in tqdm(
    holdouts_generator.split(X, y), # returns a generator over indices of train/test set
    total=number_of_splits,
    desc="Computing holdouts"
):
  X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
  y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]
  performance = train_decision_tree(
      X_train.values,
      X_test.values,
      y_train.values,
      y_test.values
  )
  all_performance += performance
  break # TODO: remove this break once the code is completed

all_performance = pd.DataFrame(all_performance)

pandas.core.frame.DataFrame